<a href="https://colab.research.google.com/github/satyearth/dijkstra/blob/main/labprogram5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
# Load a smaller dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
text = "\n".join(dataset["train"]["text"][:1000])  # First 1000 lines

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
text

'\n = Valkyria Chronicles III = \n\n\n Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n\n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for 

In [5]:
# Tokenize and limit vocabulary size
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [6]:
#The tokenizer.word_counts attribute provides a collections.
#OrderedDict containing the frequency of each word in the dataset after fitting the Tokenizer.
#This dictionary shows how many times each word appears in the text.
tokenizer.word_counts

OrderedDict([('valkyria', 54),
             ('chronicles', 39),
             ('iii', 17),
             ('senjō', 5),
             ('no', 40),
             ('3', 34),
             ('unrecorded', 1),
             ('japanese', 6),
             ('戦場のヴァルキュリア3', 3),
             ('lit', 5),
             ('of', 1533),
             ('the', 3494),
             ('battlefield', 8),
             ('commonly', 5),
             ('referred', 5),
             ('to', 1034),
             ('as', 380),
             ('outside', 16),
             ('japan', 5),
             ('is', 229),
             ('a', 883),
             ('tactical', 3),
             ('role', 24),
             ('playing', 12),
             ('video', 10),
             ('game', 90),
             ('developed', 13),
             ('by', 295),
             ('sega', 5),
             ('and', 1354),
             ('media', 6),
             ('vision', 8),
             ('for', 418),
             ('playstation', 9),
             ('portable', 4),
      

In [7]:
#The tokenizer.word_index attribute in Keras'
#Tokenizer is a dictionary that maps each unique word in the text to a unique integer index.
#It is created after the Tokenizer is fitted on a dataset using the fit_on_texts() method.

#Key Details:
#Keys: The unique words found in the corpus.
#Values: The corresponding integer index for each word, where the most frequent word receives index 1, the second most frequent word receives index 2, and so on.

tokenizer.word_index

{'the': 1,
 'of': 2,
 'and': 3,
 'in': 4,
 'to': 5,
 'a': 6,
 'was': 7,
 'for': 8,
 'with': 9,
 'that': 10,
 'as': 11,
 "'s": 12,
 'were': 13,
 'by': 14,
 'on': 15,
 'is': 16,
 'at': 17,
 'from': 18,
 'are': 19,
 'it': 20,
 'gods': 21,
 'an': 22,
 'be': 23,
 'he': 24,
 'her': 25,
 'his': 26,
 'their': 27,
 'which': 28,
 'she': 29,
 'had': 30,
 'or': 31,
 'this': 32,
 '–': 33,
 'but': 34,
 'not': 35,
 'one': 36,
 'deities': 37,
 'they': 38,
 'other': 39,
 'two': 40,
 'also': 41,
 'aircraft': 42,
 'its': 43,
 'all': 44,
 'game': 45,
 'who': 46,
 'after': 47,
 'first': 48,
 'has': 49,
 'ga': 50,
 'have': 51,
 'these': 52,
 'god': 53,
 'more': 54,
 'some': 55,
 "'": 56,
 'during': 57,
 'egyptian': 58,
 'time': 59,
 'when': 60,
 'there': 61,
 '1': 62,
 'team': 63,
 'year': 64,
 'most': 65,
 'into': 66,
 'little': 67,
 'could': 68,
 'new': 69,
 'while': 70,
 'than': 71,
 'been': 72,
 'world': 73,
 'only': 74,
 'many': 75,
 'season': 76,
 'such': 77,
 'valkyria': 78,
 'being': 79,
 '2': 80,
 

In [8]:
total_words

7641

In [9]:
# Prepare sequences
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

In [10]:
# Pad sequences
max_sequence_len = 20
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [11]:
input_sequences

array([[   0,    0,    0, ...,    0,   78,  118],
       [   0,    0,    0, ...,   78,  118,  322],
       [   0,    0,    0, ...,    0, 1241,  112],
       ...,
       [  10,  110,  860, ...,  677,  110, 3610],
       [ 110,  860,  569, ...,  110, 3610,    7],
       [ 860,  569, 7639, ..., 3610,    7, 7640]], dtype=int32)

In [12]:
# Predictors and labels
predictors, labels = input_sequences[:, :-1], input_sequences[:, -1]
labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [13]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [14]:
predictors

array([[   0,    0,    0, ...,    0,    0,   78],
       [   0,    0,    0, ...,    0,   78,  118],
       [   0,    0,    0, ...,    0,    0, 1241],
       ...,
       [  10,  110,  860, ...,    1,  677,  110],
       [ 110,  860,  569, ...,  677,  110, 3610],
       [ 860,  569, 7639, ...,  110, 3610,    7]], dtype=int32)

In [15]:
# Define a smaller model
model = Sequential([
    Embedding(total_words, 50, input_length=max_sequence_len - 1),
    LSTM(100),
    Dense(total_words, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

In [18]:
# Train with fewer epochs
history = model.fit(predictors, labels, epochs=1, verbose=1)

1424/1424 ━━━━━━━━━━━━━━━━━━━━ 51s 36ms/step - accuracy: 0.1147 - loss: 6.6138


In [19]:
import numpy as np

# Function to generate predictions
def generate_next_word(model, tokenizer, input_text, max_sequence_len=10):
    # Step 1: Tokenize and convert input_text to sequences
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]

    # Step 2: If the sequence is too long, truncate it to fit the model's input length
    if len(input_sequence) > max_sequence_len - 1:
        input_sequence = input_sequence[-(max_sequence_len - 1):]

    # Step 3: Pad the sequence if it's shorter than max_sequence_len - 1
    input_sequence = np.pad(input_sequence, (max_sequence_len - 1 - len(input_sequence), 0), mode='constant')

    # Step 4: Reshape input_sequence for LSTM model (as batch_size, time_steps)
    input_sequence = np.array(input_sequence).reshape(1, max_sequence_len - 1)

    # Step 5: Make prediction
    prediction = model.predict(input_sequence)

    # Step 6: Get the index of the word with the highest probability
    predicted_index = np.argmax(prediction)

    # Step 7: Convert the index back to a word
    predicted_word = tokenizer.index_word[predicted_index]

    return predicted_word

# Example Usage:
input_text = "The quick brown fox"
predicted_word = generate_next_word(model, tokenizer, input_text)
print(f"Input: {input_text}")
print(f"Predicted next word: {predicted_word}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
Input: The quick brown fox
Predicted next word: in
